In [1]:
import os
from sqlalchemy import create_engine
import pandas as pd
import pickle
from urllib.parse import quote_plus
from dotenv import load_dotenv

load_dotenv()

password = os.getenv("POSTGRES_PASSWORD")
username = os.getenv("POSTGRES_USERNAME")
host = os.getenv("POSTGRES_HOST")
port = os.getenv("POSTGRES_PORT")
db = os.getenv("POSTGRES_DB")

password_encoded = quote_plus(password)

# Load your pickle file
with open("/Users/evro/Documents/code/python/fetch/data/validated/users.pkl", "rb") as f:
    users = pickle.load(f)
with open("/Users/evro/Documents/code/python/fetch/data/validated/receipts.pkl", "rb") as f:
    receipts = pickle.load(f)
with open("/Users/evro/Documents/code/python/fetch/data/validated/receipt_item_data.pkl", "rb") as f:
    receipt_items = pickle.load(f)
with open("/Users/evro/Documents/code/python/fetch/data/validated/brands.pkl", "rb") as f:
    brands = pickle.load(f)

# Create a SQLAlchemy engine tied to the same connection
engine = create_engine(f"postgresql+psycopg2://{username}:{password_encoded}@{host}:{port}/{db}")

# Write the DataFrames to PostgreSQL tables.
users.to_sql('users', engine, index=False, if_exists='replace')
receipts.to_sql('receipts', engine, index=False, if_exists='replace')
receipt_items.to_sql('receipt_items', engine, index=False, if_exists='replace')
brands.to_sql('brands', engine, index=False, if_exists='replace')

167

In [44]:
# What are the top 5 brands by receipts scanned for most recent month?
q1_a = """
    SELECT
        b.name AS brand_name,
        COUNT(r.receipt_id) AS receipts_scanned
    FROM receipts r
    JOIN receipt_items ri ON r.receipt_id = ri.receipt_id
    JOIN brands b ON ri.brand_code = b.brand_code
    WHERE
        r.date_scanned >  CURRENT_DATE - INTERVAL '1 months'
    GROUP BY b.name
    ORDER BY receipts_scanned DESC
    LIMIT 5;
"""

pd.read_sql_query(q1_a, engine)

,brand_name,receipts_scanned


In [19]:
# It appears that no data is available for the last month so let's check the scanned dates
# If the latest month has low data, it mixes older brands
q1_b = """
select
    b.name AS brand_name,
    COUNT(r.receipt_id) AS receipts_scanned,
    TO_CHAR(r.date_scanned, 'YYYY-MM') AS last_scanned_date
from receipts r
join receipt_items ri on r.receipt_id = ri.receipt_id
JOIN brands b ON ri.brand_code = b.brand_code
group by TO_CHAR(r.date_scanned, 'YYYY-MM'), b.name
order by TO_CHAR(r.date_scanned, 'YYYY-MM') DESC, COUNT(r.receipt_id) desc
limit 5;
"""
pd.read_sql_query(q1_b, engine)

,brand_name,receipts_scanned,last_scanned_date
0,Viva,1,2021-02
1,Pepsi,93,2021-01
2,Kleenex,88,2021-01
3,KNORR,79,2021-01
4,Doritos,77,2021-01


In [57]:
# If the latest month doesn’t have 5 brands, expand to the previous month
q1_c = """
WITH month_ranked AS (
    SELECT
        TO_CHAR(r.date_scanned, 'YYYY-MM') AS month_year,
        b.name AS brand_name,
        COUNT(r.receipt_id) AS receipts_scanned,
        ROW_NUMBER() OVER (PARTITION BY TO_CHAR(r.date_scanned, 'YYYY-MM') ORDER BY COUNT(r.receipt_id) DESC) AS rank_per_month
    FROM receipts r
    JOIN receipt_items ri ON r.receipt_id = ri.receipt_id
    JOIN brands b ON ri.brand_code = b.brand_code
    WHERE r.receipt_id IS NOT NULL
    GROUP BY month_year, b.name
    ORDER BY month_year DESC, receipts_scanned DESC
),
latest_month AS (
    SELECT month_year FROM month_ranked ORDER BY month_year DESC LIMIT 1
),
expanded_data AS (
    SELECT * FROM month_ranked WHERE month_year = (SELECT month_year FROM latest_month)
    UNION ALL
    SELECT * FROM month_ranked WHERE month_year < (SELECT month_year FROM latest_month)
    ORDER BY month_year DESC, receipts_scanned DESC
    LIMIT 5
)
SELECT brand_name, receipts_scanned, month_year
FROM expanded_data;
"""
pd.read_sql_query(q1_c, engine)

,brand_name,receipts_scanned,month_year
0,Viva,1,2021-02
1,Pepsi,93,2021-01
2,Kleenex,88,2021-01
3,KNORR,79,2021-01
4,Doritos,77,2021-01


In [58]:
# How does the ranking of the top 5 brands by receipts scanned for the recent month compare to the ranking for the previous month?
q2 = """
WITH current_month AS (
    SELECT
        b.name AS brand_name,
        COUNT(r.receipt_id) AS receipts_scanned
    FROM receipts r
    JOIN receipt_items ri ON r.receipt_id = ri.receipt_id
    JOIN brands b ON ri.brand_code = b.brand_code
    WHERE
        r.date_scanned >= DATE_TRUNC('month', CURRENT_DATE - INTERVAL '1 month')
        AND r.date_scanned < DATE_TRUNC('month', CURRENT_DATE)
    GROUP BY b.name
    ORDER BY receipts_scanned DESC
    LIMIT 5
),
previous_month AS (
    SELECT
        b.name AS brand_name,
        COUNT(r.receipt_id) AS receipts_scanned
    FROM receipts r
    JOIN receipt_items ri ON r.receipt_id = ri.receipt_id
    JOIN brands b ON ri.brand_code = b.brand_code
    WHERE
        r.date_scanned >= DATE_TRUNC('month', CURRENT_DATE - INTERVAL '2 month')
        AND r.date_scanned < DATE_TRUNC('month', CURRENT_DATE - INTERVAL '1 month')
    GROUP BY b.name
    ORDER BY receipts_scanned DESC
    LIMIT 5
)
SELECT
    cm.brand_name AS current_brand,
    cm.receipts_scanned AS current_count,
    pm.brand_name AS previous_brand,
    pm.receipts_scanned AS previous_count
FROM current_month cm
FULL OUTER JOIN previous_month pm ON cm.brand_name = pm.brand_name;

"""

pd.read_sql_query(q2, engine)

,current_brand,current_count,previous_brand,previous_count


In [68]:
# When considering average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

# There are not and "accepted" values in rewardsReceiptStatus, replacing with finished.
q3 = """
SELECT
    rewards_receipt_status,
    AVG(total_spent) AS avg_spent
FROM receipts
WHERE
    rewards_receipt_status IN ('FINISHED', 'REJECTED') AND total_spent > 0
GROUP BY rewards_receipt_status;
"""

pd.read_sql_query(q3, engine)

,rewards_receipt_status,avg_spent
0,REJECTED,24.355147
1,FINISHED,81.167694


In [70]:
# When considering total number of items purchased from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

# There are not and "accepted" values in rewardsReceiptStatus, replacing with finished.
q4 = """
SELECT
    rewards_receipt_status,
    SUM(purchased_item_count) AS total_items
FROM receipts
WHERE
    rewards_receipt_status IN ('FINISHED', 'REJECTED')
GROUP BY rewards_receipt_status;

"""
pd.read_sql_query(q4, engine)

,rewards_receipt_status,total_items
0,REJECTED,173.0
1,FINISHED,8184.0


In [66]:
# Which brand has the most spend among users who were created within the past 6 months?
q5 = """
    SELECT
        b.name AS brand_name,
        SUM(r.total_spent) AS total_spend
    FROM users u
    JOIN receipts r ON u.user_id = r.user_id
    JOIN receipt_items ri ON r.receipt_id = ri.receipt_id
    JOIN brands b ON ri.brand_code = b.brand_code
    WHERE
        u.created_date >= CURRENT_DATE - INTERVAL '6 months'
    GROUP BY b.name
    ORDER BY total_spend DESC
    LIMIT 1;
"""

pd.read_sql_query(q5, engine)

,brand_name,total_spend


In [111]:
# Which brand has the most transactions among users who were created within the past 6 months?
q6 = """
    SELECT
        b.name AS brand_name,
        COUNT(r.receipt_id) AS transaction_count
    FROM users u
    JOIN receipts r ON u.user_id = r.user_id
    JOIN receipt_items ri ON r.receipt_id = ri.receipt_id
    JOIN brands b ON ri.brand_code = b.brand_code
    WHERE
        u.created_date >= CURRENT_DATE - INTERVAL '6 months'
    GROUP BY b.name
    ORDER BY transaction_count DESC
    LIMIT 1;
"""
pd.read_sql_query(q6, engine)

,brand_name,transaction_count
